In [60]:
import requests as req
from bs4 import BeautifulSoup as bs
from concurrent.futures import ThreadPoolExecutor
import pymysql

In [61]:
 # yahoo_stock_URL
url = "https://tw.stock.yahoo.com/quote/8070.TW"


# 用 requests 的 get 方法把網頁抓下來
res = req.get(url) 

# 指定 lxml 作為解析器
soup = bs(res.text, "lxml") 


In [62]:
# 嘗試找到股票名稱.價格.漲跌幅
id = soup.select_one('span.C\(\$c-icon\)')
title = soup.select('h1')[1]
price = soup.select_one('.Fz\(32px\)')
percent = soup.select('.Fz\(20px\)')[0]
status = ""
print(id)
print(title)
print(price)
print(percent)


<span class="C($c-icon) Fz(24px) Mend(20px)">8070</span>
<h1 class="C($c-link-text) Fw(b) Fz(24px) Mend(8px)">長華*</h1>
<span class="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)">57.9</span>
<span class="Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c) C($c-trend-down)"><span class="Mend(4px) Bds(s)" style="border-color:#00ab5e transparent transparent transparent;border-width:9px 6.5px 0 6.5px"></span>2.00</span>


In [63]:
# 如果 main-0-QuoteHeader-Proxy id 的 div 裡有 C($c-trend-down) 的 class
# 表示狀態為下跌
if soup.select_one('#main-0-QuoteHeader-Proxy .C\(\$c-trend-down\)'):
    status= '-'
# 如果 main-0-QuoteHeader-Proxy id 的 div 裡有 C($c-trend-up) 的 class
# 表示狀態為上漲
elif soup.select_one('#main-0-QuoteHeader-Proxy .C\(\$c-trend-up\)'):
    status= '+'
else:
    status = ''
    
# 印出結果
print(f'{id.get_text()} {title.get_text()} : {price.get_text()} ( {status}{percent.get_text()} )')  

8070 長華* : 57.9 ( -2.00 )


In [64]:
#想抓取的股票網址清單
stock_urls = [
  'https://tw.stock.yahoo.com/quote/8070',
  'https://tw.stock.yahoo.com/quote/2352',
  'https://tw.stock.yahoo.com/quote/8150',
  'https://tw.stock.yahoo.com/quote/2344',
  'https://tw.stock.yahoo.com/quote/6120',
  'https://tw.stock.yahoo.com/quote/6548',

]

def getStock(stock_urls):
    res = req.get(stock_urls) 
    soup = bs(res.text, "lxml")
    id = soup.select_one('span.C\(\$c-icon\)')
    title = soup.select('h1')[1]
    price = soup.select_one('.Fz\(32px\)')
    percent = soup.select('.Fz\(20px\)')[0]
    status = ""
    try:
        if soup.select_one('#main-0-QuoteHeader-Proxy .C\(\$c-trend-down\)'):
            status= '-'
        elif soup.select_one('#main-0-QuoteHeader-Proxy .C\(\$c-trend-up\)'):
            status= '+'
        else:
            status = ''  
    except Exception as e:
        print(f"Error in parsing status for {stock_urls}: {e}")
        status = ''
    
    stock_data = {
        'id': id.get_text(),
        'name': title.get_text(),
        'price': price.get_text(),
        'status': status,
        'percent': percent.get_text()
    }

    return f'{stock_data["id"]} {stock_data["name"]} : {stock_data["price"]} ( {stock_data["status"]}{stock_data["percent"]} )'

# 使用執行緒池來並行抓取股票資訊
with ThreadPoolExecutor() as executor:
    results = list(executor.map(getStock, stock_urls))  # 收集所有結果

# 印出所有結果
for result in results:
    print(result)
    


8070 長華* : 57.9 ( -2.00 )
2352 佳世達 : 35.60 ( -0.20 )
8150 南茂 : 37.40 ( -0.15 )
2344 華邦電 : 23.45 ( +0.05 )
6120 達運 : 14.45 ( 0.00 )
6548 長科* : 39.35 ( -0.45 )
